This notebook follows from the file `hetero_2_3_analyse_the_annual_ts.py`.
# Read the data

In [1]:
import h5py
import numpy as np

filename = r"C:\SUSTech\datasets_of_graduation_project\hetero_outputs\decadely_slopes.h5"

with h5py.File(filename, 'r') as f:
    ALPHAS = f['alphas'][:]
    DECADES = f['decades'][:]
    LATS = f['latitudes'][:]
    LONS = f['longitudes'][:]
    DECADELY_SLOPES = f['decadely_slopes'][:]
    
print("The shape of slopes is: (len(ALPHAS), len(DECADES), len(LATS), len(LONS)) =", DECADELY_SLOPES.shape)

The shape of slopes is: (len(ALPHAS), len(DECADES), len(LATS), len(LONS)) = (5, 8, 60, 144)


# Visualizations

In [2]:
print(ALPHAS)
print(DECADES)
print(LATS)
print(LONS)
DECADELY_SLOPES[0, 0]

[90 75 50 25 10]
[1945 1955 1965 1975 1985 1995 2005 2015 2025]
[90.   89.5  89.   88.5  88.   87.5  87.   86.5  86.   85.5  85.   84.5
 84.   83.5  83.   82.5  82.   81.5  81.   80.5  80.   79.5  79.   78.5
 78.   77.5  77.   76.5  76.   75.5  74.75 74.25 73.75 73.25 72.75 72.25
 71.75 71.25 70.75 70.25 69.75 69.25 68.75 68.25 67.75 67.25 66.75 66.25
 65.75 65.25 64.75 64.25 63.75 63.25 62.75 62.25 61.75 61.25 60.75 60.  ]
[-180.   -177.5  -175.   -172.5  -170.   -167.5  -165.   -162.5  -160.
 -157.5  -155.   -152.5  -150.   -147.5  -145.   -142.5  -139.75 -137.25
 -134.75 -132.25 -129.75 -127.25 -124.75 -122.25 -119.75 -117.25 -114.75
 -112.25 -109.75 -107.25 -104.75 -102.25  -99.5   -97.    -94.5   -92.
  -89.5   -87.    -84.5   -82.    -79.5   -77.    -74.5   -72.    -69.5
  -67.    -64.5   -62.    -59.25  -56.75  -54.25  -51.75  -49.25  -46.75
  -44.25  -41.75  -39.25  -36.75  -34.25  -31.75  -29.25  -26.75  -24.25
  -21.75  -19.    -16.5   -14.    -11.5    -9.     -6.5    -4.    

array([[0.00612109, 0.00612109, 0.00612109, ..., 0.00612109, 0.00612109,
        0.00612109],
       [0.00434311, 0.0051965 , 0.00599643, ..., 0.00642201, 0.00531635,
        0.00436494],
       [0.00370927, 0.00412468, 0.00463109, ..., 0.00444724, 0.00404219,
        0.00376846],
       ...,
       [0.1404924 , 0.14319846, 0.15435901, ..., 0.09193689, 0.12035652,
        0.13469812],
       [0.11051414, 0.11978871, 0.16195512, ..., 0.10392918, 0.09952763,
        0.10762292],
       [0.09651637, 0.10512029, 0.15110306, ..., 0.08233846, 0.07740386,
        0.09656946]], dtype=float32)

In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import matplotlib.colors as mcolors
import numpy as np

lons, lats = np.meshgrid(LONS, LATS)  # 生成规则网格

# 假设已有：ALPHAS, overall_slopes, lats, lons

for alpha_idx, alpha in enumerate(ALPHAS):
    N_COLS = 4
    N_ROWS = int(np.ceil(len(ALPHAS) / N_COLS))
    fig, axes = plt.subplots(N_ROWS, N_COLS, figsize=(5 * N_COLS, 5 * N_ROWS), subplot_kw={'projection': None})

    # 统一颜色归一化范围
    slope_min = np.min(DECADELY_SLOPES[alpha_idx].flatten())
    norm = mcolors.Normalize(vmin=-np.abs(slope_min), vmax=np.abs(slope_min))
    cmap = "coolwarm"

    # 将经纬度转换为 Basemap 使用的坐标
    m = Basemap(projection="npstere", boundinglat=60, lon_0=0, resolution="l")
    x, y = m(lons, lats)

    # 如果 axes 只有一行，axes 需要转为二维数组统一处理
    # axes = np.array(axes).reshape(-1, 2)

    decadely_slopes_alpha = DECADELY_SLOPES[alpha_idx]

    for decade_idx, decade in enumerate(DECADES[:-1]):
        row_idx, col_idx = divmod(decade_idx, N_COLS)
        ax = axes[row_idx, col_idx]
        
        m_ax = Basemap(projection="npstere", boundinglat=60, lon_0=0, resolution="l", ax=ax)
        decadely_slopes_alpha_decade = decadely_slopes_alpha[decade_idx]

        x, y = m_ax(lons, lats)
        c = m_ax.pcolormesh(x, y, decadely_slopes_alpha_decade, cmap=cmap, norm=norm, shading="auto")

        m_ax.drawcoastlines()
        m_ax.drawcountries()
        m_ax.drawparallels(np.arange(60, 91, 10))
        m_ax.drawmeridians(np.arange(-180, 181, 30))

        ax.set_title(f"{decade}s")

    # 添加共享颜色条
    cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])  # [left, bottom, width, height]
    fig.colorbar(c, cax=cbar_ax, ticks=[-np.abs(slope_min), 0, np.abs(slope_min)], orientation='vertical')
    cbar_ax.set_ylabel("°C/year")

    fig.suptitle("Yearly increase of 2m temperature percentiles (°C/yr) for alpha = {:.2f}".format(alpha), fontsize=16, y=0.95)
    plt.tight_layout(rect=[0, 0, 0.9, 0.96])  # 留出空间给标题和颜色条
    
    fig.savefig(r"C:\SUSTech\datasets_of_graduation_project\hetero_outputs\fig_outputs\dt0sa\heatmaps_of_2mtemp_annual" + str(alpha) + ".png", dpi=300, bbox_inches='tight')
    plt.close(fig)  # 关闭图形以释放内存

C:\Users\15523\AppData\Local\Temp\ipykernel_19284\4026849795.py:37: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  c = m_ax.pcolormesh(x, y, decadely_slopes_alpha_decade, cmap=cmap, norm=norm, shading="auto")
C:\Users\15523\AppData\Local\Temp\ipykernel_19284\4026849795.py:37: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  c = m_ax.pcolormesh(x, y, decadely_slopes_alpha_decade, cmap=cmap, norm=norm, shading="auto")
C:\Users\15523\AppData\Local\Temp\ipykernel_19284\4026849795.py:37: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonica

In [4]:
DECADELY_SLOPES[0, :, 0, 0]

array([ 0.00612109,  0.00738877,  0.00010487, -0.00909202, -0.00669963,
        0.00431482, -0.01037191, -0.00882957], dtype=float32)

In [5]:
print(np.min(DECADELY_SLOPES[0].flatten()))
print(np.min(DECADELY_SLOPES[1].flatten()))
print(np.min(DECADELY_SLOPES[2].flatten()))
print(np.min(DECADELY_SLOPES[3].flatten()))
print(np.min(DECADELY_SLOPES[4].flatten()))

-0.5050644
-0.4844094
-0.6382792
-0.9792903
-1.2086023
